In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier

df = pd.read_csv("/content/dataset.csv")

feature_cols = [
    'PTS_x', 'REB_x', 'AST_x', 'STL_x', 'BLK_x', 'MIN_x',
    'PTS_pg_x', 'REB_pg_x', 'AST_pg_x', 'STL_pg_x', 'BLK_pg_x', 'MIN_pg_x'
]

X = df[feature_cols]
y = df["breakout?"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    random_state=42,
    eval_metric="logloss"
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Precision: 0.512
Recall: 0.053
F1 Score: 0.096

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.98      0.87      1322
           1       0.51      0.05      0.10       396

    accuracy                           0.77      1718
   macro avg       0.64      0.52      0.48      1718
weighted avg       0.72      0.77      0.69      1718

